# EHSD Lab Exercise 1

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from get_tl_model import get_tl_model

In [ ]:
def load_ltspice(path = "../ltspice/circuit_model.asc"):
    """
    Loads the LTSpice model and parses the component values.

    Returns:
    lines (list[str]): list of strings containing all lines from the LTSpice model
    param_lines (list[str]): list of strings containing all .param lines from the LTSpice model
    param_idx (list[int]): list of integers containing the line index of each .param line
    """

    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    param_idx = []
    param_lines = []
    for i, line in enumerate(lines):
        if '.param' in line:
            param_idx.append(i)
            param_lines.append(line)

    print("------ Current Parameters ------")
    for line in param_lines:
        values = line.split('.param')[1].strip()
        print(values)

    return lines, param_idx, param_lines

def edit_ltspice(lines, param_lines, param_idx, component_values,
                 path: str = "../ltspice/circuit_model.asc"):
    """
    Edits the LTSpice model with the new component values.
    
    Args:
    lines (list[str]): list of strings containing all lines from the LTSpice model
    param_lines (list[str]): list of strings containing all .param lines from the LTSpice model
    param_idx (list[int]): list of integers containing the line index of each .param line
    component_values (dict): dictionary containing the new component values
    path (str): path to the LTSpice model file
    """

    for i, line in enumerate(param_lines):
        mod_line = line.split('.param')[1].strip().split(' ')

        for key, value in component_values.items():
            for j, param in enumerate(mod_line):
                param = param.split('=')[0]
                if key == param:
                    mod_line[j] = key + '=' + value
        mod_line = ' '.join(mod_line)
        
        lines[param_idx[i]] = lines[param_idx[i]].split('.param')[0] + '.param ' + mod_line + '\n'

    param_lines = []
    for i, line in enumerate(lines):
        if '.param' in line:
            param_lines.append(line)

    print("\n------ New Parameters ------")
    for line in param_lines:
        values = line.split('.param')[1].strip()
        print(values)

    usr_input = input("Write? (y/n): ")
    if usr_input == 'y':
        with open("../ltspice/circuit_model.asc", 'w', encoding='utf-8') as f:
            f.writelines(lines)

In [ ]:
# Case 1, side-by-side microstrip
case = 1
tl_model_params = get_tl_model(case=case)
print(f"------ TL Model Parameters (Case {case}) ------")
for key, value in tl_model_params.items():
    if isinstance(value, np.ndarray):
        print(f"{key}:\n", np.round(value, 2))
    else:
        print(f"{key}:\t", "{:.2e}".format(value))

ibis_output = {
    'R_pkg_1': '93.36m',
    'R_pkg_2': '100.13m',
    'R_pkg_3': '118.19m',
    'C_pkg_1': '0.15918p',
    'C_pkg_2': '0.14031p',
    'C_pkg_3': '0.15918p',
    'L_pkg_1': '1.33430n',
    'L_pkg_2': '1.44133n',
    'L_pkg_3': '1.73153n'
}

ibis_input = {
    'R_pkg_1': '95.98m',
    'R_pkg_2': '93.15m',
    'R_pkg_3': '101.95m',
    'C_pkg_1': '0.13711p',
    'C_pkg_2': '0.12663p',
    'C_pkg_3': '0.14411p',
    'L_pkg_1': '1.38443n',
    'L_pkg_2': '1.34060n',
    'L_pkg_3': '1.47927n'
}

component_values = {
    # Driver
    'C1': '6.04p',      # CONST: C_comp OUT (IBIS)
    'R1': '10.9',       # CONST: HW TEST BOARD MODELLING (PDF)
    # Receiver
    'C4': '4.21p',      # CONST: C_comp IN (IBIS)
    # QFN Package Output
    'L1': ibis_output[f'L_pkg_{case}'],   # L_pkg (IBIS)
    'R2': ibis_output[f'R_pkg_{case}'],   # R_pkg (IBIS)
    'C2': ibis_output[f'C_pkg_{case}'],   # C_pkg (IBIS)
    # QFN Package Input
    'L2': ibis_input[f'L_pkg_{case}'],    # L_pkg (IBIS)
    'R7': ibis_input[f'R_pkg_{case}'],    # R_pkg (IBIS)
    'C3': ibis_input[f'C_pkg_{case}'],    # C_pkg (IBIS)
    # Board Resistors
    'R3': '39',
    'R4': '1k',
    'R5': '50',
    'R6': '1k',
    'R8': '1k',
    # TL Model
    'L11': f"{tl_model_params['L_seg']}n",
    'C11': f"{tl_model_params['C_seg']}p",
    'L21': f"{tl_model_params['L_seg']}n",
    'C21': f"{tl_model_params['C_seg']}p",
    'Cm': f"{tl_model_params['Cm_seg']}p",
    'K': f"{tl_model_params['K']}"
}

lines, param_idx, param_lines = load_ltspice()
edit_ltspice(lines, param_lines, param_idx, component_values)

In [ ]:
for case in range(1, 4):
    tl_model_params = get_tl_model(case=case)
    print(f"------ TL Model Parameters (Case {case}) ------")
    for key, value in tl_model_params.items():
        if isinstance(value, np.ndarray):
            print(f"{key}:\n", np.round(value, 2))
        else:
            print(f"{key}:\t", "{:.2e}".format(value))